In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import PreTrainedTokenizer, PreTrainedModel, DataCollatorWithPadding
from datasets import load_dataset, Features, Value
# from datasets_helper import *

/uufs/chpc.utah.edu/common/home/u1283221/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_dataset = load_dataset(
    "csv",
    data_files="../data/multiplication-v2/multiply-primed-1.csv",
    features=Features(
        {
            "priming_instruction": Value("string"),
            "priming_question": Value("string"),
            "priming_answer": Value("string"),
            "instruction": Value("string"),
            "example_question": Value("string"),
            "example_answer": Value("string"),
            "question": Value("string"),
        }
    ),
)["train"]

In [3]:
raw_dataset[0]

{'priming_instruction': 'Carry the digit from the tens place.',
 'priming_question': 'The result of 59 // 10 is what?',
 'priming_answer': '5',
 'instruction': 'Multiply two numbers.',
 'example_question': 'The result of 36799 * 57443 is what?',
 'example_answer': '2113844957',
 'question': 'The result of 0 * 0 is what?'}

In [7]:
from prompt_template import PromptTemplate as PT

def get_full_llama_2_chat_prompt_v2(
    instruction,
    question,
    example_question,
    example_answer,
    priming_instruction=None,
    priming_question=None,
    priming_answer=None,
):
    pt = PT()
    if (
        (priming_instruction is not None and priming_instruction != "")
        and (priming_question is not None and priming_question != "")
        and (priming_answer is not None and priming_answer != "")
    ):
        pt.add_user_message(f"Q: {priming_instruction}\n{priming_question}")
        pt.add_model_reply(f"A: {priming_answer}")
    pt.add_user_message(f"Q: {instruction}\n{example_question}")
    pt.add_model_reply(f"A: {example_answer}")
    pt.add_user_message(f"Q: {instruction}\n{question}")
    return pt.build_prompt()

In [8]:
sample = raw_dataset[0]
print(get_full_llama_2_chat_prompt_v2(**sample))

<s>Q: Carry the digit from the tens place.
The result of 59 // 10 is what? [/INST] A: 5 </s>[INST] Q: Multiply two numbers.
The result of 36799 * 57443 is what? [/INST] A: 2113844957 </s>[INST] Q: Multiply two numbers.
The result of 0 * 0 is what? [/INST]
